In [1]:
from qaravan.tensorQ import *
from qaravan.core import *

In [2]:
# testing environments 
n = 4
sv = random_sv(n)

gate_list = [H(0), CNOT([1,0], n), CNOT([2,1], n), CNOT([3,2], n)]
circ = Circuit(gate_list, n)
sim = StatevectorSim(circ)
final_state = sim.run(progress_bar=False).reshape(2**n)
overlap_a = sv.conj().T @ final_state

for gate_idx in range(len(circ)):
    env, mat = sv_environment(circ, sv, gate_idx)
    overlap_b = np.trace(env @ mat)
    print(np.allclose(overlap_a, overlap_b))

True
True
True
True


In [9]:
# testing environment based state prep
skeleton = [(0,1), (2,3), (1,2), (0,1), (2,3)]
target_sv = random_sv(4)

context = RunContext(
    progress_interval=50,
    max_iter=500,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=100,
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::500]

Step 0 at time 2025-04-17T12:44:31: cost = 0.014518341460622142
[Checkpoint saved at step 0]


Step 50 at time 2025-04-17T12:44:31: cost = 4.546377771352805e-05
Step 100 at time 2025-04-17T12:44:32: cost = 1.928842922005458e-05
[Checkpoint saved at step 100]
Step 150 at time 2025-04-17T12:44:32: cost = 1.0643510892305308e-05
Step 200 at time 2025-04-17T12:44:32: cost = 6.3644923727901315e-06
[Checkpoint saved at step 200]
Step 250 at time 2025-04-17T12:44:33: cost = 3.947853291830228e-06
Step 300 at time 2025-04-17T12:44:33: cost = 2.5012390205692014e-06
[Checkpoint saved at step 300]
Step 350 at time 2025-04-17T12:44:33: cost = 1.6072516333398568e-06
Step 400 at time 2025-04-17T12:44:34: cost = 1.0434273124415583e-06
[Checkpoint saved at step 400]
Step 450 at time 2025-04-17T12:44:34: cost = 6.826945708748866e-07
Max iterations reached with cost 4.5315715280214164e-07


(400,
 [np.float64(0.7697355942897102),
  np.float64(3.5324236868916614e-05),
  np.float64(1.427824062327776e-05),
  np.float64(7.260102290573833e-06),
  np.float64(3.984786012711794e-06),
  np.float64(2.2521653082474558e-06),
  np.float64(1.3048048875807794e-06)])

In [10]:
# resuming from previous run
context = RunContext(
    progress_interval=50,
    max_iter=1000,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=100,
    resume=True
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::500]

[Resuming from checkpoint: step 400]
Step 400 at time 2025-04-17T12:45:05: cost = 1.0345397410960189e-06
[Checkpoint saved at step 400]


Step 450 at time 2025-04-17T12:45:06: cost = 6.769721763166814e-07
Step 500 at time 2025-04-17T12:45:07: cost = 4.45687649897053e-07
[Checkpoint saved at step 500]
Step 550 at time 2025-04-17T12:45:07: cost = 2.948404260738613e-07
Step 600 at time 2025-04-17T12:45:07: cost = 1.958067951113307e-07
[Checkpoint saved at step 600]
Step 650 at time 2025-04-17T12:45:08: cost = 1.3044584179500163e-07
Step 700 at time 2025-04-17T12:45:08: cost = 8.712412136180347e-08
[Checkpoint saved at step 700]
Step 750 at time 2025-04-17T12:45:08: cost = 5.83105397300443e-08
Step 800 at time 2025-04-17T12:45:09: cost = 3.909223789122507e-08
[Checkpoint saved at step 800]
Step 850 at time 2025-04-17T12:45:09: cost = 2.6244282858023382e-08
Step 900 at time 2025-04-17T12:45:09: cost = 1.76388588180032e-08
[Checkpoint saved at step 900]
Step 950 at time 2025-04-17T12:45:09: cost = 1.1866123639237003e-08
Max iterations reached with cost 8.052135735958643e-09


(900,
 [np.float64(0.7697355942897102),
  np.float64(3.5324236868916614e-05),
  np.float64(1.427824062327776e-05),
  np.float64(7.260102290573833e-06),
  np.float64(3.984786012711794e-06),
  np.float64(2.2521653082474558e-06),
  np.float64(1.3048048875807794e-06),
  np.float64(7.635725286503359e-07),
  np.float64(4.5315715280214164e-07),
  np.float64(2.6998358881868967e-07),
  np.float64(1.623800165440059e-07),
  np.float64(9.774663023431884e-08),
  np.float64(5.925262369643747e-08),
  np.float64(3.5887696814285164e-08),
  np.float64(2.185747027816376e-08)])